In [ ]:
#soruce of BeatAML2 drug response data: https://biodev.github.io/BeatAML2/
# Citation: Bottomly, D., Long, N., Schultz, A. R., Kurtz, S. E., Tognon, C. E., Johnson, K., … & Tyner, J. W. (2022). Integrative analysis of drug response and clinical outcome in acute myeloid leukemia. Cancer Cell, 40(8), 850-864. Article Link


# curve loading and drug name harmonizing

import re
import difflib
import requests
import pandas as pd

beataml_path = "other/beataml_probit_curve_fits_v4_dbgap.txt"
other_datasets = ["CTRPv2", "GDSC1", "GDSC2", "CTRPv1", "CCLE"]

data = pd.read_csv(beataml_path, sep="\t")
beataml_inhibitors = pd.Series(sorted(data["inhibitor"].dropna().unique()), name="drug_name_beataml")

joined = []
for dataset in other_datasets:
    df = pd.read_csv(f"../{dataset}/drug_names.csv")
    for col in ["drug_name", "compound_name", "name"]:
        if col in df.columns:
            joined.extend(df[col].dropna().astype(str).tolist())
            break
joined = sorted(set(joined))

SYNONYMS = {
    "azd1152-hqpa": "barasertib",
    "azd1152 hqpa": "barasertib",
    "dovitinib (chir-258)": "dovitinib",
    "chir-258": "dovitinib",
    "r406": "fostamatinib",
    "cyt387": "momelotinib",
    "tg101348": "fedratinib",
    "ink-128": "sapanisertib",
    "pp242": "torkinib",
    "roscovitine (cyc-202)": "roscovitine",
    "volasertib (bi-6727)": "volasertib",
    "vatalanib (ptk787)": "vatalanib",
    "tofacitinib (cp-690550)": "tofacitinib",
    "gsk-2879552": "gsk2879552",
    "gsk-1904529a": "gsk1904529a",
    "gsk-1838705a": "gsk1838705a",
    "ly-333531": "ly333531",
    "kn93": "kn-93",
    "go6976": "go-6976",
    "h-89": "h89",
    "pd98059": "pd-98059",
    "pd153035": "pd-153035",
    "ym-155": "ym155",
    "bi-6727": "volasertib",
    "cp-690550": "tofacitinib",
    "ptk787": "vatalanib",
    "baiclein": "baicalein",
    "vargetef": "nintedanib",
    "gdc-0941": "pictilisib",
    "jq1": "jq1"
}

STOP = {
    "inhibitor","inhibitors","maleate","hydrochloride","hcl","sodium",
    "monohydrate","hydrate","salt","free","base","acid","analog","agonist",
    "antagonist","mk","bi","cp","azd","gsk","jnj","ly","tg","mln","pd","pp",
    "vx","ki","sr","at","xmd","pf","bms","bay"
}

GENERIC_BLOCK = {
    "akt inhibitor i","akt inhibitor ii","akt inhibitor iii","akt inhibitor iv","akt inhibitor v","akt inhibitor vi","akt inhibitor vii","akt inhibitor viii",
    "jnk inhibitor i","jnk inhibitor ii","jnk inhibitor viii",
    "jak inhibitor i","mek1/2 inhibitor","ampk inhibitor","nf-kb activation inhibitor","p38 map kinase inhibitor","syk inhibitor"
}

def norm(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("α","alpha").replace("β","beta").replace("γ","gamma").replace("δ","delta")
    s = re.sub(r"[®™]", "", s)
    s = s.replace("/", " ").replace("\\", " ")
    s = re.sub(r"[,\(\)\[\]\.]", " ", s)
    s = re.sub(r"\s+", " ", s)
    s = s.replace("–","-").replace("—","-").replace("−","-")
    return s.strip()

def toks(s: str):
    s = norm(s)
    s = re.sub(r"[^a-z0-9\- ]", " ", s)
    return [t for t in s.split() if t and t not in STOP]

def tkey(s: str) -> str:
    return " ".join(toks(s))

def is_generic_label(s: str) -> bool:
    n = norm(s)
    if n in GENERIC_BLOCK:
        return True
    return " inhibitor" in n and not re.search(r"\b[a-z]{2,}\d{2,}", n)

def is_mixture_label(s: str) -> bool:
    n = norm(s)
    if "mol" in n:
        return True
    hyph = re.findall(r"\b[A-Za-z]{2,}\-?\d{2,}\b", s)
    return len(hyph) >= 2 and "-" in s

def match_one(name, pool, token_thresh=0.6, fuzzy_thresh=0.87, used_targets=None):
    n = norm(name)
    tk = tkey(name)
    if is_generic_label(name) or is_mixture_label(name):
        return None
    if n in SYNONYMS:
        syn = SYNONYMS[n]
        for c in pool:
            if norm(c) == norm(syn) or tkey(c) == tkey(syn):
                if used_targets is None or c not in used_targets:
                    return c
    for c in pool:
        if norm(c) == n or tkey(c) == tk:
            if used_targets is None or c not in used_targets:
                return c
    for c in pool:
        if is_mixture_label(c):
            continue
        cn = norm(c)
        if n in cn or cn in n:
            if used_targets is None or c not in used_targets:
                return c
    tn = set(toks(name))
    if tn:
        best_c, best_j = None, 0.0
        for c in pool:
            if is_mixture_label(c):
                continue
            tc = set(toks(c))
            if not tc:
                continue
            j = len(tn & tc) / len(tn | tc)
            if j > best_j:
                best_j, best_c = j, c
        if best_c and best_j >= token_thresh:
            if used_targets is None or best_c not in used_targets:
                return best_c
    cand = {c: norm(c) for c in pool if not is_mixture_label(c)}
    if not cand:
        return None
    best = max(cand.keys(), key=lambda c: difflib.SequenceMatcher(None, cand[c], n).ratio())
    score = difflib.SequenceMatcher(None, cand[best], n).ratio()
    if score >= fuzzy_thresh:
        if used_targets is None or best not in used_targets:
            return best
    return None

def split_variants(s: str):
    s = s.strip()
    out = [s]
    m = re.search(r"^(.*?)[\s]*\(([^)]+)\)[\s]*$", s)
    if m:
        out += [m.group(1).strip(), m.group(2).strip()]
    more = []
    for v in list(out):
        if "-" in v:
            more += [v.replace("-", " "), v.replace("-", "")]
        if " " in v:
            more += [v.replace(" ", "-"), v.replace(" ", "")]
    out += more
    seen, res = set(), []
    for v in out:
        k = v.lower()
        if v and k not in seen:
            seen.add(k)
            res.append(v)
    return res

ALIASES_RESOLVE = {
    "azd1152-hqpa": ["Barasertib","AZD2811","AZD1152 HQPA"],
    "dovitinib (chir-258)": ["Dovitinib","CHIR-258","TKI258"],
    "ink-128": ["Sapanisertib","MLN0128","INK128"],
    "roscovitine (cyc-202)": ["Seliciclib","Roscovitine","CYC-202"],
    "vatalanib (ptk787)": ["Vatalanib","PTK787"],
    "ym-155": ["Sepantronium","YM155"],
    "h-89": ["H 89","H89"],
    "go6976": ["Gö 6976","GO 6976","GO6976"],
    "vargetef": ["Nintedanib","Vargatef","Nintedanib esylate"],
    "baiclein": ["Baicalein"],
    "gdc-0941": ["Pictilisib","GDC-0941"]
}

def pubchem_cid(name: str) -> str:
    qs = [name] + split_variants(name) + ALIASES_RESOLVE.get(norm(name), [])
    seen, qlist = set(), []
    for q in qs:
        k = q.lower()
        if k not in seen:
            seen.add(k)
            qlist.append(q)
    for q in qlist:
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{requests.utils.quote(q)}/cids/JSON"
        try:
            r = requests.get(url, timeout=20)
            if r.status_code == 200:
                j = r.json()
                ids = j.get("IdentifierList", {}).get("CID", [])
                if ids:
                    return str(ids[0])
        except requests.RequestException:
            pass
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/description/JSON"
    for q in qlist[:5]:
        try:
            r = requests.get(url, params={"name": q}, timeout=20)
            if r.status_code == 200:
                j = r.json()
                infos = j.get("InformationList", {}).get("Information", [])
                for info in infos:
                    cid = info.get("CID")
                    if cid:
                        return str(cid)
        except requests.RequestException:
            pass
    return ""

used_targets = set()
rows = []
for bname in beataml_inhibitors:
    m = match_one(bname, joined, used_targets=used_targets)
    final_name = bname if m is None else m
    if m is not None:
        used_targets.add(m)
    cid = pubchem_cid(final_name)
    rows.append({"drug_name_beataml": bname, "drug_name": final_name, "pubchem_id": cid})

df_map = pd.DataFrame(rows)

df_map["pubchem_id"][df_map["pubchem_id"].isna()] = df_map["drug_name_beataml"][df_map["pubchem_id"].isna()]


In [ ]:
#create response file
response_data = data.merge(df_map, left_on="inhibitor", right_on="drug_name_beataml", how="left")
response_data["LN_IC50"] = response_data["ic50"]
response_data.drop(columns=["ic50", "drug_name_beataml", "status", "paper_inclusion"], inplace=True)
response_data["cell_line_name"] = response_data["dbgap_subject_id"].astype(str)
response_data.to_csv("BeatAML2.csv", index=False)


In [ ]:
# create raw dose response file for curve curator
raw_data = pd.read_csv("other/beataml_wv1to4_raw_inhibitor_v4_dbgap.txt", sep="\t")
required_columns = ["dose", "response", "sample", "drug", "replicate"]
raw_data["dose"] = raw_data["well_concentration"]
raw_data["response"] = raw_data["normalized_viability"]
raw_data["sample"] = raw_data["dbgap_subject_id"].astype(str)
#map drug name
raw_data["drug"] = [df_map.set_index("drug_name_beataml").at[name, "pubchem_id"] if name in df_map["drug_name_beataml"].values else name for name in raw_data["inhibitor"] ]
raw_data.to_csv("BeatAML2_raw.csv", index=False)

In [ ]:
import pandas as pd
raw_data = pd.read_csv("BeatAML2_c_raw.csv", dtype={"drug": str})
raw_data.loc[raw_data.inhibitor=="MEK1/2 Inhibitor", "drug"] = "MEK1/2 Inhibitor"
raw_data.loc[raw_data.inhibitor=="JQ1", "drug"] = "JQ12"
def clean_pubchem_id(x):
    if x.endswith(".0"):
        return x[:-2]
    return x

raw_data["drug"] = raw_data["drug"].apply(clean_pubchem_id)
raw_data.to_csv("BeatAML2_c_raw.csv", index=False)


In [ ]:
# create gene expression feature file
gene_exp = pd.read_csv("other/beataml_waves1to4_norm_exp_dbgap.txt", sep="\t", index_col=0)

gene_exp_processed = gene_exp
gene_exp_processed =  gene_exp_processed.reset_index().drop(columns=["stable_id", "description", "biotype"]).set_index("display_label")
gene_exp_processed = gene_exp_processed.T
map_rna_id_to_cl_name = {rna_ind: sub_ind for sub_ind, rna_ind in zip(response_data["cell_line_name"], response_data["dbgap_rnaseq_sample"])}
#remap index
gene_exp_processed.index = [map_rna_id_to_cl_name.get(ind, pd.NA) for ind in gene_exp_processed.index]
gene_exp_processed =  gene_exp_processed.drop(pd.NA)
gene_exp_processed.to_csv("gene_expression.csv", index=True)

In [ ]:
# create cell line names file
cell_line_names = response_data.drop_duplicates("cell_line_name")[["cell_line_name"]]
cell_line_names["Cellosaurus_id"] = pd.NA
cell_line_names["tissue"] = "Blood"
cell_line_names.to_csv("cell_line_names.csv", index=False)

In [ ]:
# create drug names file
drug_names = df_map[["drug_name_beataml", "drug_name", "pubchem_id"]]
#if pubchemid is NA, use drug_name
drug_names["pubchem_id"] = drug_names["pubchem_id"].replace("", pd.NA)
drug_names["pubchem_id"] = drug_names["pubchem_id"].replace(" ", pd.NA)

drug_names["pubchem_id"] = drug_names["pubchem_id"].fillna(drug_names["drug_name"])

drug_names.to_csv("drug_names.csv", index=False)

In [ ]:
#create fingerprints
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# df has columns: pubchem_id, canonical_smiles
drug_smiles = pd.read_csv("drug_smiles.csv")

df = drug_smiles.copy()

out_dir = "drug_fingerprints"
os.makedirs(out_dir, exist_ok=True)

def morgan_bits(smiles, n_bits, radius=2):
    m = Chem.MolFromSmiles(smiles)
    if m is None:
        return np.full(n_bits, np.nan)  # keep shape even if invalid
    fp = AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

sizes = [64, 128, 256, 512, 1024, 2048]

for n in sizes:
    print(f"computing {n}-bit fingerprints")
    # compute and stack into a DataFrame (rows = bit positions, cols = pubchem_id)
    fp_matrix = {}
    for pid, smi in zip(df["pubchem_id"], df["canonical_smiles"]):
        fp_matrix[pid] = morgan_bits(smi, n)
    fp_df = pd.DataFrame(fp_matrix)  # index 0..n-1, columns = pubchem_id
    fp_df.to_csv(f"{out_dir}/pubchem_id_to_demorgan_{n}_map.csv", index=False)


In [ ]:
# rename genes in gene expression file  to correct symbols
ge = pd.read_csv("gene_expression.csv", index_col=0)
genes = list(ge.columns)
new_gene_names = {
    "AARS": "AARS1",
    "EPRS": "EPRS1",
    "FAM57A": "TLCD3A",
    "FAM69A": "DIPK1A",
    "H2AFV": "H2AZ2",
    "HIST1H2BK": "H2BC12",
    "HIST2H2BE": "H2BC21",
    "KIAA0100": "BLTP2",
    "KIAA0355": "GARRE1",
    "NARFL": "CIAO3",
    "PAPD7": "TENT4A",
    "SKIV2L": "SKIC2",
    "TSTA3": "GFUS",
    "WDR61": "SKIC8",
    "WRB": "GET1",
}
new_genes = [new_gene_names.get(g, g) for g in genes]
ge.columns = new_genes
landmark_genes  = pd.read_csv("gene_lists/landmark_genes.csv", index_col=0)
set(landmark_genes["Symbol"]) - set(new_genes)
#save again
ge.index.name = "cell_line_name"
ge.to_csv("gene_expression.csv", index=True)

In [ ]:
# correct pubchem ids in BeatAML2.csv
import pandas as pd

d = pd.read_csv("BeatAML2.csv")
d["pubchem_id"] = d["pubchem_id"].astype(str).str.replace(".0", "", regex=False)

d.loc[d["drug_name"] == "MEK1/2 Inhibitor", "pubchem_id"] = "MEK1/2 Inhibitor"
d.loc[d["drug_name"] == "JQ12", "pubchem_id"] = "JQ12"

d.to_csv("BeatAML2.csv", index=False)
